In [1]:
%%time
import sys
sys.path.append('/home/ubuntu/reps/skip-thoughts')

import skipthoughts

# this takes a while
model = skipthoughts.load_model()

Loading model parameters...
Compiling encoders...

Using gpu device 0: GRID K520 (CNMeM is disabled)
/home/ubuntu/reps/Theano/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)



Loading tables...
Packing up...


In [20]:
# test model
r = skipthoughts.encode(model, ['Which type of activity is directly responsible for geothermal energy?', 'weather', 'wind', 'solar', 'volcanic'])

from scipy.spatial.distance import cosine
import numpy as np

assert np.argmax([cosine(r[0], r[i+1]) for i in range(4)]) == 3  # correct answer is volcanic

1
11


In [32]:
import codecs
import os
from scipy.spatial.distance import cosine
import numpy as np
DATA_FOLDER = '/home/ubuntu/data'
TRAINING_FILE = os.path.join(DATA_FOLDER, 'training_set.tsv')
VALIDATION_FILE = os.path.join(DATA_FOLDER, 'validation_set.tsv')
SUBMISSION_FILE = '/home/ubuntu/thereisnoanswer.txt'
order = ['A', 'B', 'C', 'D']

In [35]:
%%time
data = []
corrects = []
with codecs.open(TRAINING_FILE, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    for row in (line.strip().split('\t') for line in f):
        qid, Q, correct, A, B, C, D = row
        data.extend([Q, A, B, C, D])
        corrects.append(correct)

correct_count = 0
wrong_count = 0
v = skipthoughts.encode(model, data)
for i in range(v.shape[0] / 5):
    my_answer = np.argmax([cosine(v[i*5], v[i*5 + k + 1]) for k in range(4)])
    print(corrects[i], order[my_answer])
    if corrects[i] == order[my_answer]:
        correct_count += 1
    else:
        wrong_count += 1
            
print('score', correct_count / (wrong_count + correct_count + 0.0))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
65
66
67
68
69
70
72
73
74
75
76
77
79
80
81
82
84
87
91
92
95
97
117
119
135
138
153
158
196
(u'C', 'B')
(u'C', 'B')
(u'D', 'A')
(u'B', 'D')
(u'B', 'C')
(u'B', 'D')
(u'A', 'D')
(u'A', 'C')
(u'D', 'D')
(u'C', 'D')
(u'D', 'B')
(u'D', 'C')
(u'A', 'A')
(u'C', 'C')
(u'B', 'A')
(u'B', 'B')
(u'C', 'B')
(u'A', 'D')
(u'B', 'A')
(u'B', 'A')
(u'C', 'A')
(u'C', 'B')
(u'C', 'C')
(u'D', 'A')
(u'A', 'D')
(u'B', 'B')
(u'D', 'C')
(u'D', 'C')
(u'B', 'C')
(u'B', 'D')
(u'B', 'B')
(u'B', 'A')
(u'B', 'A')
(u'A', 'C')
(u'A', 'D')
(u'D', 'B')
(u'C', 'A')
(u'B', 'A')
(u'B', 'B')
(u'D', 'C')
(u'D', 'D')
(u'A', 'A')
(u'A', 'C')
(u'A', 'A')
(u'C', 'A')
(u'C', 'D')
(u'A', 'B')
(u'A', 'D')
(u'A', 'A')
(u'B', 'B')
(u'C', 'C')
(u'B', 'D')
(u'A', 'A')
(u'D', 'A')
(u'B', 'B')
(u'D', 'A')
(u'A', 'D')
(u'A', 'C')
(u'B', 'A')
(u'C', 'D')
(u'A',

In [34]:
%%time
# prepare submission

i=0
data = []
qids = []
with codecs.open(VALIDATION_FILE, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    for row in (line.strip().split('\t') for line in f):
        qid, Q, A, B, C, D = row
        data.extend([Q, A, B, C, D])
        qids.append(qid)
        
sv = skipthoughts.encode(model, data)    
with open(SUBMISSION_FILE, 'w') as w:
    w.write('id,correctAnswer\n')
    for i in range(sv.shape[0] / 5):
        my_answer = np.argmax([cosine(sv[i*5], sv[i*5 + k + 1]) for k in range(4)])
        w.write('%s,%s\n' % (qids[i], order[my_answer]))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
56
58
59
60
62
64
65
67
69
71
72
129
75
80
82
83
89
94
165
105
CPU times: user 37.4 s, sys: 13.2 s, total: 50.6 s
Wall time: 50.6 s
